
# AirCanada Hallucination Evals — End‑to‑End (LLM‑first)

This notebook runs the entire pipeline Rune asked for:

1) **Generate scenarios** (LLM‑synthesized; template fallback)  
2) **Run the multi‑turn evals** against a toy support bot (with groundedness filter & escalation)  
3) **Judge** with two independent rules and **escalate** on disagreement/low‑confidence  
4) **Aggregate & visualize** coverage  
5) Inspect **flagged examples** and **re‑run** a scenario to show the multi‑turn transcript

> **Prereqs:** Run this at your repo root (`aiuc_aircanada_eval/`).  
> **Providers:** Set `PROVIDER=openai` + `OPENAI_API_KEY`, or `PROVIDER=anthropic` + `ANTHROPIC_API_KEY`.


## 0) Setup (first run only)

In [ ]:
# If you have not installed these yet, uncomment and run:
# !pip install openai anthropic matplotlib pandas

import os, sys, json, pathlib, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

BASE = Path.cwd()
print("Working dir:", BASE)

# Show provider hint (set these in your environment or here before running step 1)
print("PROVIDER =", os.environ.get("PROVIDER", "(not set; default 'openai')"))
print("OPENAI_API_KEY set? ", bool(os.environ.get("OPENAI_API_KEY")))
print("ANTHROPIC_API_KEY set? ", bool(os.environ.get("ANTHROPIC_API_KEY")))


## 1) Generate scenarios (LLM‑first; template fallback)

In [ ]:
import subprocess, shlex
from pathlib import Path

# Prefer LLM mode
cmd_llm = [sys.executable, "src/generate_scenarios.py", "--per-cell", "2", "--mode", "llm"]
print("Running:", " ".join(cmd_llm))
ret = subprocess.run(cmd_llm)
scen_path = Path("evals/scenarios.jsonl")

# Fallback to template if generation failed or file missing
if ret.returncode != 0 or not scen_path.exists():
    print("LLM generation failed or scenarios missing; falling back to template.")
    cmd_tpl = [sys.executable, "src/generate_scenarios.py", "--per-cell", "2", "--mode", "template"]
    subprocess.check_call(cmd_tpl)

print("Scenarios written to:", scen_path)
print("\nPreview (first 3 lines):")
print("\n".join(scen_path.read_text().splitlines()[:3]))


## 2) Run the eval pipeline

In [ ]:
import subprocess, shlex, json
subprocess.check_call([sys.executable, "src/run.py"])

agg_path = Path("evals/aggregate.json")
print("Aggregate:")
print(agg_path.read_text())


## 3) Visualize coverage (heatmap)

In [ ]:
import subprocess, shlex
from IPython.display import Image, display

subprocess.check_call([sys.executable, "src/viz.py"])
img_path = Path("images/coverage_heatmap.png")
display(Image(filename=str(img_path)))


## 4) Inspect flagged examples

In [ ]:
import csv, pandas as pd
df = pd.read_csv("evals/results.csv")
flags = df[df["label"] != "safe"]
print("Total flags:", len(flags))
flags.head(10)


## 5) Re‑run a scenario to view the multi‑turn transcript + verdicts

In [ ]:
from sut import SupportBot
from judge import judge_dialogue, judge_dialogue_alt
import json

policy = Path("policy/policy.txt").read_text()
bot = SupportBot(policy)

# Load the first scenario for demo
scen = json.loads(open("evals/scenarios.jsonl").read().splitlines()[0])
history = []
for turn in scen['dialogue']:
    if turn['role'] == 'user':
        # append user turn
        history.append(turn)
        # bot replies
        reply = bot.answer(history, {"attack": scen["attack"], "use_case": scen["use_case"]})
        history.append({"role":"assistant","content": reply})

print("ATTACK:", scen['attack'], "| USE_CASE:", scen['use_case'])
print("\n--- Transcript ---")
for m in history:
    role = m['role'].upper()
    print(f"{role}: {m['content']}")

print("\n--- Judges ---")
print("Judge A:", judge_dialogue(history, policy))
print("Judge B:", judge_dialogue_alt(history, policy))


## 6) Groundedness KPI (citation rate)

In [ ]:
# Approximate: count how many example_bot_utterance rows include a [§N] citation
df = pd.read_csv("evals/results.csv")
has_cite = df["example_bot_utterance"].astype(str).str.contains("\[§")
rate = has_cite.mean()
print(f"Citation presence in final utterance: {rate:.2%}  (target: ~100% after guardrail)")


## 7) Generate buyer-facing summary (Markdown)

In [ ]:
from pathlib import Path
import sys
sys.path.append("src")
from report_gen import generate_buyer_report
print("Report path:", generate_buyer_report(Path.cwd()))

In [ ]:
# ## 8) Residual risk bands (use case / tactic / combo)
import pandas as pd
from risk import compute_risk_summaries


df = pd.read_csv("evals/results.csv")
risk = compute_risk_summaries(df)
print("By use case:")
pd.DataFrame(risk["by_use_case"]).sort_values("risk_index", ascending=False).head(10)
